**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data files**

In [2]:
FOLDER = "annotation_tss_pol2"

In [3]:
FD_REG = file.path(
    FD_RES, 
    "results", 
    "region", 
    "annotation_enhancer_merge")

FD_ANT = file.path(
    FD_RES, 
    "results", 
    "region", 
    FOLDER)

In [4]:
fdiry  = FD_REG
fnames = dir(fdiry)
for(fname in fnames){print(fname)}

[1] "description.tsv"
[1] "enhancer_merge.concat.starrmpra_crispr_atac.active.bed.gz"
[1] "enhancer_merge.concat.starrmpra_crispr_atac.active.merge.bed.gz"
[1] "enhancer_merge.concat.starrmpra_crispr_atac.total.bed.gz"
[1] "enhancer_merge.concat.starrmpra_crispr_atac.total.merge.bed.gz"
[1] "enhancer_table.starrmpra_crispr_atac.active.merge.tsv"
[1] "enhancer_table.starrmpra_crispr_atac.total.merge.tsv"
[1] "region_active"
[1] "region_total"
[1] "summary"
[1] "tmp"


In [5]:
fdiry  = FD_ANT
fnames = dir(fdiry)
for(fname in fnames){print(fname)}

[1] "description.tsv"
[1] "K562.TSS.selected_by_highest_Pol2_signal.bed"
[1] "K562.TSS.selected_by_highest_Pol2_signal.bed.gz"
[1] "K562.TSS.selected_by_highest_Pol2_signal.filtered_by_RNAseq_TPM.bed.gz"


In [6]:
fdiry = file.path(FD_REG, "region_active", FOLDER)
fnames = dir(fdiry)
for(fname in fnames){print(fname)}

[1] "peak.annotation.K562.TSS.selected_by_highest_Pol2_signal.bed.gz"
[1] "peak.annotation.K562.TSS.selected_by_highest_Pol2_signal.filtered_by_RNAseq_TPM.bed.gz"


## Import data

In [7]:
### set file paths
fdiry = FD_REG
fname = "description.tsv"
fpath = file.path(fdiry, fname)

### read table
dat = read_tsv(fpath, show_col_types = FALSE)
vec = dat$Name
vec[1:3] = paste(vec[1:3], "Peak", sep="_")

### assign and show
vec_txt_cnames_reg = vec
dat

Name,Description
<chr>,<chr>
Chrom,Chromosome
Start,Start
End,End
Assay_Type,Assay type
Assay_Label,Assay label


In [8]:
### set file paths
fdiry = FD_ANT
fname = "description.tsv"
fpath = file.path(fdiry, fname)

### read table
dat = read_tsv(fpath, show_col_types = FALSE)
vec = dat$Name

### assign and show
vec_txt_cnames_ant = vec
dat

Name,Description
<chr>,<chr>
Chrom,Chromosome
Start,TSS position
End,TSS position
Name,Gene name
Score,"highest level of pol2 chip seq (ENCFF914WIS.bigWig) at [TSS-500, TSS+500] among TSS isoform"


In [12]:
### init: set column names
cnames = c(
    paste(vec_txt_cnames_reg, sep="_"),
    paste(vec_txt_cnames_ant, sep="_"),
    "Overlap"
)

### init: set file path
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "annotation_enhancer_merge",
    "region_active",
    FOLDER)
fname  = paste("peak.annotation", "bed.gz", sep="*")
fglob  = file.path(fdiry, fname)
fpaths = Sys.glob(fglob)
fnames = basename(fpaths)

### init: annotation and label
annotations = c("TSS_Pol2", "TSS_Pol2_RNAseq")

names(fnames)      = annotations
names(annotations) = annotations

print(fnames)

                                                                                TSS_Pol2 
                       "peak.annotation.K562.TSS.selected_by_highest_Pol2_signal.bed.gz" 
                                                                         TSS_Pol2_RNAseq 
"peak.annotation.K562.TSS.selected_by_highest_Pol2_signal.filtered_by_RNAseq_TPM.bed.gz" 


In [17]:
### Import data
lst = lapply(annotations, function(annotation){
    ### set file path
    fname = fnames[annotation]
    fpath = file.path(fdiry, fname)
    
    ### read data
    dat = read_tsv(fpath, col_names = cnames, show_col_types = FALSE)
    #dat = dat %>% dplyr::mutate(
    #    Name_Peak  = paste0(Chrom_Peak, ":", Start_Peak, "-", End_Peak),
    #    Annotation = annotation,
    #    Label      = Category,
    #    Region     = paste0(Chrom, ":", Start, "-", End),
    #    Value      = Score,
    #    Note       = paste0(Name, ":", Category)
    #)
    ### arrange columns
    dat = dat %>% dplyr::mutate(
        Name_Peak  = paste0(Chrom_Peak, ":", Start_Peak, "-", End_Peak),
        Annotation = annotation,
        Label      = "TSS",
        Region     = paste0(Chrom, ":", Start, "-", End),
        Value      = Score,
        Note       = Name
    )
    return(dat)
})

### assign and show
lst_peak_annot_import = lst
print(length(lst))
cat("=========================\n")
print(names(lst))
cat("=========================\n")
head(lst[[1]])

[1] 2
[1] "TSS_Pol2"        "TSS_Pol2_RNAseq"


Chrom_Peak,Start_Peak,End_Peak,Assay_Type,Assay_Label,Chrom,Start,End,Name,Score,Overlap,Name_Peak,Annotation,Label,Region,Value,Note
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
chr1,17237,17772,ATAC,ATAC,chr1,17436,17437,MIR6859-1,9.43812,1,chr1:17237-17772,TSS_Pol2,TSS,chr1:17436-17437,9.43812,MIR6859-1
chr1,17237,17772,ATAC,ATAC,chr1,17436,17437,MIR6859-2,9.43812,1,chr1:17237-17772,TSS_Pol2,TSS,chr1:17436-17437,9.43812,MIR6859-2
chr1,17237,17772,ATAC,ATAC,chr1,17436,17437,MIR6859-3,9.43812,1,chr1:17237-17772,TSS_Pol2,TSS,chr1:17436-17437,9.43812,MIR6859-3
chr1,17237,17772,ATAC,ATAC,chr1,17436,17437,MIR6859-4,9.43812,1,chr1:17237-17772,TSS_Pol2,TSS,chr1:17436-17437,9.43812,MIR6859-4
chr1,28903,29613,ATAC,ATAC,chr1,29370,29371,WASH7P,0.00023,1,chr1:28903-29613,TSS_Pol2,TSS,chr1:29370-29371,0.00023,WASH7P
chr1,628759,629259,ENCODE-E2G_Prediction,"ENCODE-E2G_Prediction:PLEKHN1,ENCODE-E2G_Prediction:RNF223,ENCODE-E2G_Prediction:SAMD11,ENCODE-E2G_Prediction:SCNN1D,ENCODE-E2G_Prediction:TNFRSF18,ENCODE-E2G_Prediction:TTLL10",chr1,629009,629010,LOC101928626,0.00023,1,chr1:628759-629259,TSS_Pol2,TSS,chr1:629009-629010,0.00023,LOC101928626


## Arrange and summarize

In [18]:
### init
lst = lst_peak_annot_import

lst = lapply(lst, function(dat){
    ### summarize
    dat = dat %>%
        dplyr::group_by(Chrom_Peak, Start_Peak, End_Peak, Name_Peak, Assay_Type, Assay_Label, Annotation, Label) %>%
        dplyr::summarise(
            Count  = n(), 
            Region = paste(Region, collapse="|"),
            Score  = mean(Value),
            Note   = paste(Note, collapse="|"),
            .groups = "drop")
    
    ### Arrange
    dat = dat %>% 
        dplyr::arrange(Chrom_Peak,  Start_Peak,  End_Peak) %>%
        dplyr::select(-Chrom_Peak, -Start_Peak, -End_Peak) %>%
        dplyr::rename("Peak" = Name_Peak)
    return(dat)
})

### assign and show
lst_peak_annot_summary = lst
print(length(lst))
cat("=========================\n")
print(names(lst))
cat("=========================\n")
head(lst[[1]])

[1] 2
[1] "TSS_Pol2"        "TSS_Pol2_RNAseq"


Peak,Assay_Type,Assay_Label,Annotation,Label,Count,Region,Score,Note
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<chr>
chr1:17237-17772,ATAC,ATAC,TSS_Pol2,TSS,4,chr1:17436-17437|chr1:17436-17437|chr1:17436-17437|chr1:17436-17437,9.4381200,MIR6859-1|MIR6859-2|MIR6859-3|MIR6859-4
chr1:28903-29613,ATAC,ATAC,TSS_Pol2,TSS,1,chr1:29370-29371,0.0002300,WASH7P
chr1:628759-629259,ENCODE-E2G_Prediction,"ENCODE-E2G_Prediction:PLEKHN1,ENCODE-E2G_Prediction:RNF223,ENCODE-E2G_Prediction:SAMD11,ENCODE-E2G_Prediction:SCNN1D,ENCODE-E2G_Prediction:TNFRSF18,ENCODE-E2G_Prediction:TTLL10",TSS_Pol2,TSS,1,chr1:629009-629010,0.0002300,LOC101928626
chr1:632439-632939,ENCODE-E2G_Prediction,"ENCODE-E2G_Prediction:AGRN,ENCODE-E2G_Prediction:CALML6,ENCODE-E2G_Prediction:CFAP74,ENCODE-E2G_Prediction:FAM87B,ENCODE-E2G_Prediction:ISG15,ENCODE-E2G_Prediction:KLHL17,ENCODE-E2G_Prediction:LINC00115,ENCODE-E2G_Prediction:LINC01128,ENCODE-E2G_Prediction:LINC01342,ENCODE-E2G_Prediction:LOC100288069,ENCODE-E2G_Prediction:LOC105378591,ENCODE-E2G_Prediction:MMEL1,ENCODE-E2G_Prediction:PLEKHN1,ENCODE-E2G_Prediction:PRKCZ,ENCODE-E2G_Prediction:RNF223,ENCODE-E2G_Prediction:SAMD11,ENCODE-E2G_Prediction:SCNN1D,ENCODE-E2G_Prediction:TNFRSF18,ENCODE-E2G_Prediction:TNFRSF4,ENCODE-E2G_Prediction:TP73,ENCODE-E2G_Prediction:TP73-AS1,ENCODE-E2G_Prediction:TPRG1L,ENCODE-E2G_Prediction:TTLL10",TSS_Pol2,TSS,1,chr1:632685-632686,0.0730961,MIR12136
chr1:777949-779437,"ASTARR,ATAC,ENCODE-E2G_Prediction,LMPRA,WSTARR","ASTARR_A,ATAC,ENCODE-E2G_Prediction:FAM87B,ENCODE-E2G_Prediction:LINC00115,ENCODE-E2G_Prediction:LINC01128,ENCODE-E2G_Prediction:LOC100288069,ENCODE-E2G_Prediction:SAMD11,LMPRA_A,WSTARR_A,WSTARR_AB",TSS_Pol2,TSS,1,chr1:778634-778635,230.5610000,LOC100288069
chr1:816774-817620,"ATAC,ENCODE-E2G_Prediction","ATAC,ENCODE-E2G_Prediction:FAM87B,ENCODE-E2G_Prediction:LINC00115,ENCODE-E2G_Prediction:LINC01128",TSS_Pol2,TSS,1,chr1:817370-817371,1.0660900,FAM87B


## Explore and check results

In [19]:
lst = lst_peak_annot_summary
lst = lapply(lst, function(dat){
    res = table(dat$Count)
    return(res)
})
print(lst)

$TSS_Pol2

    1     2     3     4     5     6     7    10 
15054  2417   133    16     3     1     2     1 

$TSS_Pol2_RNAseq

   1    2    3    5    7 
8864 1171   32    1    1 



## Save results

In [21]:
fdiry = file.path(
    FD_REG, 
    "region_active", 
    "summary")

lst = lst_peak_annot_summary
for (idn in names(lst)){
    fname = paste("peak.summary", idn,"tsv", sep=".")
    fpath = file.path(fdiry, fname)
    print(fname)
    flush.console()
    
    dat = lst[[idn]]
    write_tsv(dat, fpath)
}

[1] "peak.summary.TSS_Pol2.tsv"
[1] "peak.summary.TSS_Pol2_RNAseq.tsv"
